In [ ]:
# Code for mounting the AWS S3 bucket(only needs running once)

from pyspark.sql.functions import *
import urllib
# define the credentials path
delta_table_path = "dbfs:/user/hive/warehouse/authentication_credentials"
# read the delta table to a Spark DataFrame

aws_keys_df = spark.read.format("delta").load(delta_table_path)

# get access key and secret access key from spark dataframe
ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']

# encode the secret access key

ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

# AWS S3 bucket name
AWS_S3_BUCKET = "user-12885f560a0b-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/mounted-user-12885f560a0b-bucket"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

# Should output True

In [ ]:
# File location and type
file_location = "s3a://user-12885f560a0b-bucket/topics/12885f560a0b.pin/partition=0/*.json" 
file_type = "json"  # Corrected file type to match the files being read
# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from mounted S3 bucket
pin_df = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location)
# Display Spark dataframe to check its content
display(pin_df)

In [ ]:
# File location and type
file_location = "s3a://user-12885f560a0b-bucket/topics/12885f560a0b.geo/partition=0/*.json" 
file_type = "json"  # Corrected file type to match the files being read
# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from mounted S3 bucket
geo_df = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location)
# Display Spark dataframe to check its content
display(geo_df)

In [ ]:
# File location and type
file_location = "s3a://user-12885f560a0b-bucket/topics/12885f560a0b.user/partition=0/*.json" 
file_type = "json"  # Corrected file type to match the files being read
# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from mounted S3 bucket
user_df = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location)
# Display Spark dataframe to check its content
display(user_df)